In [ ]:
#!pip install clustergram pandas_profiling scipy sklearn statsmodels IPython dtale matplotlib rpy2 seaborn shap

In [ ]:
#put in ~/.bashrc
#LD_PRELOAD="/mnt/distvol/R/4.1.2/lib64/R/lib/LibR.so"

In [ ]:
#from fracdiff import fdiff
#import urbangrammar-graphics as ugg
%matplotlib inline
import os
from clustergram import Clustergram
from concurrent.futures import wait, ALL_COMPLETED
from dask.distributed import Client
from dask.distributed import as_completed
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from numpy import absolute
from numpy import arange
from numpy import mean
from numpy import std
from pandas import read_csv
from pandas_profiling import ProfileReport
#from rpy2.robjects import pandas2ri
from pmdarima.utils import diff_inv
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
from rpy2.rinterface_lib import openrlib
from scipy import stats
from scipy.cluster.vq import vq
from scipy.spatial.distance import cdist, pdist
from scipy.special import boxcox, inv_boxcox
from scipy.stats import f
from sklearn import preprocessing
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import *
#from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing import scale
from sklearn.utils import as_float_array
from statsmodels.formula.api import ols
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import OLSInfluence
import IPython
import concurrent.futures
import dask.dataframe as dd
import datetime
import dtale
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pingouin as pg
import pmdarima
import pycorrelate
import random
import re
import rpy2
import rpy2.robjects as ro
import rpy2.situation
import scipy
import seaborn as sn
import shap
import sklearn
import sklearn.linear_model
import statsmodels.api as sm
import statsmodels.tools
import sys
import time

In [ ]:
#c = get_config()
libpath = os.environ.get('LD_LIBRARY_PATH', '')
os.environ['LD_LIBRARY_PATH'] = (
    rpy2.situation.r_ld_library_path_from_subprocess(openrlib.R_HOME) +
    libpath
)

In [ ]:
def formula_from_cols(df, y):
    return y + ' ~ ' + ' + '.join([col for col in df.columns if not col==y])

def testNormal (x):    
    
    k2, p = stats.normaltest(x)
    alpha = .001
    #print("p = {:g}".format(p))    
    if p < alpha:  # null hypothesis: x comes from a normal distribution
        #print(p)
        #print(alpha)
        print("The null hypothesis can be rejected")
        xt, _ = stats.yeojohnson(x)
        #xt, _ = stats.boxcox(x)        
        print(_)
        xt = pd.DataFrame(xt)
        
        return _, pd.DataFrame(xt).set_index(x.index)
    else:
        print("The null hypothesis cannot be rejected")    
        return 1, pd.DataFrame(x)

def inverse_boxcox (data, lambdas):
    power = PowerTransformer(method='yeo-johnson')
    power.lambdas_ = lambdas.values
    return(power.inverse_transform([data]))
    #return inv_boxcox(data, lambdas.values)
    
def transform_boxcox_l(data, l_):
    transformed = pd.DataFrame()

    for i in range(0,len(data.columns)):
        #print(i)
        if l_.iloc[i].values == 1:
            inner_scale = data.iloc[:,i]            
        else:
            inner_scale = pd.DataFrame(stats.yeojohnson((data.iloc[:,i]), lmbda=l_.iloc[i].values))
            
        inner_scale.index = data.index
        transformed = pd.concat([transformed,inner_scale],axis=1)
        
    transformed.columns = data.columns
    return transformed

def transform_boxcox (data):
    transformed = pd.DataFrame()
    transformed_lambdas = pd.DataFrame()

    for i in range(0,len(data.columns)):
        l, inner_scale = testNormal(data.iloc[:,i])
        inner_scale.set_index(data.index)

        transformed_lambdas = pd.concat([transformed_lambdas,pd.DataFrame(pd.Series(l))],axis=0)
        transformed = pd.concat([transformed,inner_scale],axis=1)
        
    transformed.columns = data.columns
    return transformed, transformed_lambdas

def inverse_yeo(og, data_, lambda_):
    values = []
    for i in range(0,len(og)):
        X = og[i]
        X_trans = data_[i]
        if X >= 0 and lambda_ == 0:
            X = exp(X_trans) - 1
        elif X >= 0 and lambda_ != 0:
            X = (X_trans * lambda_ + 1) ** (1 / lambda_) - 1
        elif X < 0 and lambda_ != 2:
            X = 1 - (-(2 - lambda_) * X_trans + 1) ** (1 / (2 - lambda_))
        elif X < 0 and lambda_ == 2:
            X = 1 - exp(-X_trans)
        
        values.append(X)
    return(pd.DataFrame(values))


def revert_yeo (og, data_, lambdas):
    reverted = pd.DataFrame()

    for i in range(0,len(data_.columns)):        
        if lambdas.iloc[i].values == 1 :
            revert = data_.iloc[:,i]
        else:
            p#ower = PowerTransformer(method='yeo-johnson')
            #power.lambdas_ = lambdas.iloc[i].values
            #revert = pd.DataFrame(power.inverse_transform([data.iloc[:,i].values]))
            #return inv_boxcox(data, lambdas.values)
            revert = pd.DataFrame(inverse_yeo(og.iloc[:,i].values,data_.iloc[:,i].values, lambdas.iloc[i].values))            
        revert.index = data_.index
        reverted = pd.concat([reverted,revert],axis=1)
        
    reverted.columns = data_.columns
    return reverted

class ZCA(BaseEstimator, TransformerMixin):
  def __init__(self, regularization=1e-5, copy=False):
      self.regularization = regularization
      self.copy = copy
  def fit(self, X, y=None):
      X = as_float_array(X, copy=self.copy)
      self.mean_ = np.mean(X, axis=0)
      X = X - self.mean_
      sigma = np.dot(X.T, X) / (X.shape[0] - 1)
      U, S, V = np.linalg.svd(sigma)
      tmp = np.dot(U, np.diag(1 / np.sqrt(S + self.regularization)))
      self.components_ = np.dot(tmp, U.T)
      return self
  def transform(self, X):
      X_transformed = X - self.mean_
      X_transformed = np.dot(X_transformed, self.components_.T)
      return X_transformed

def crosscorrelation(x, y, maxlag, mode='corr'):
    """
    Cross correlation with a maximum number of lags.

    `x` and `y` must be one-dimensional numpy arrays with the same length.

    This computes the same result as
        numpy.correlate(x, y, mode='full')[len(a)-maxlag-1:len(a)+maxlag]

    The return vaue has length 2*maxlag + 1.
    """
    py = np.pad(y.conj(), 2*maxlag, mode='constant')
    T = np.lib.stride_tricks.as_strided(py[2*maxlag:], shape=(2*maxlag+1, len(y) + 2*maxlag),
                   strides=(-py.strides[0], py.strides[0]))
    px = np.pad(x, maxlag, mode='constant')
    if mode == 'dot':       # get lagged dot product
        return T.dot(px)
    elif mode == 'corr':    # gets Pearson correlation
        return (T.dot(px)/px.size - (T.mean(axis=1)*px.mean())) / \
               (np.std(T, axis=1) * np.std(px)) 
    
def ret_ccf(npa_):
    y_name = npa_[0]
    x_name = npa_[1]
    data = cleaned.loc[training]
    y = np.array(data.iloc[:,data.columns==y_name]).ravel()
    
    x = np.array(data.iloc[:,data.columns==x_name]).ravel()
    #print(x)
    #ccf = statsmodels.tsa.stattools.ccf(x,y)
    ccf = crosscorrelation(x,y, ccf_max_lag, mode='corr')
    #print(ccf)
    return([y_name,x_name,ccf])

def train(partition):
    est = LinearRegression()
    est.fit(partition[New_Names].values, partition['target'])
    return est


def regress (df):
    lr = LinearRegression()

    variables=df.columns
    target = variables[0]
    
    temp = pd.concat([pd.DataFrame(df[target]),df.loc[:, ~df.columns.isin([target])]],axis=1)

    name = str(target)+'.csv'
    #print(name)
    
    s_f_s = sfs(lr, 
              k_features=len(df.columns)-1, 
              forward=True, 
              floating=False, 
              scoring='neg_mean_squared_error',
              n_jobs=-1,
              cv=10)
    
    s_f_s_f = s_f_s.fit(temp.loc[:, ~temp.columns.isin([target])], pd.DataFrame(temp[target]))
    
    #temp.to_csv(name)
    return(s_f_s_f.get_metric_dict())
    '''
def nv_diff_sets(v_of_i,dataset,f_casts):

  s_=sndif_[which(colnames(raw)==var_of_int)]
  d_=ndif_[which(colnames(raw)==var_of_int)]
  
  startRow = c()
  for (r in rownames(dataset[1:d_,,drop=FALSE])):
    startRow = c(startRow,which(rownames(raw)==r))
  
  data_ = c(na.omit(c(dataset[,var_of_int], f_casts)))
  
  if(s_==0):
    inv_d = diffinv(data_,differences=d_,xi=raw[startRow,var_of_int])
  else:  
    inv_d = diffinv(diffinv(data_,differences = d_, xi=raw[startRow,var_of_int]), differences = s_,xi=raw[startRow:(startRow+season-1),var_of_int])
    
  return(inv_d)
'''

def lagpad(x, k):
    length=np.full(abs(k), np.NaN)
    #print(length)
    #k=k-1
    if (k>0):
        result = np.concatenate([length,x[0:(len(x)-k)]])
    elif (k<0):
        result= np.concatenate([(x[abs(k):(len(x))]),length])
    else:
        result= x
    return(result)

def lag(data):
    return lagpad(data,1)

def sndif_(npa_):
    data = raw_int[npa_[0]]
    return(pmdarima.arima.nsdiffs(data.dropna(),m=npa_[1]))

def ndif_(npa_):
    data = raw_int[npa_[0]]
    return(pmdarima.arima.ndiffs(data.dropna()))

def clientFunction(function_name,npa):
    client = Client('192.168.3.100:8786',timeout=3)
    future = client.map(function_name,npa)

    results = []
    for f in as_completed(future):
        if(f.status=="error"):
            results.append("error")
        else:
            results.append(f.result())   

    client.close()

    return results

In [ ]:
raw = pd.read_csv("all_data.csv",index_col=0)
raw.index = pd.to_datetime(raw.index)

#fillna(method='bfill')
raw_int = raw.interpolate(method='time').dropna()


In [ ]:
#delta = (raw_int-raw_int.shift()).dropna()
#raw_delta = (raw_int - raw_int.apply(lag,0)).dropna()
#raw_delta.head()

#raw_delta.isna().sum().sum()

In [ ]:

#for i in range(0,len(raw_int.columns)):
        
#np.max(sndif)

In [ ]:
#raw_int.diff().dropna().apply(pmdarima.arima.nsdiffs(m=4))

sndif = []

season = 4
maxn = season

npa = []

for s in range(0,len(raw_int.columns)):
    npa.append([raw_int.columns[s],maxn, ])
    
sndif = clientFunction(sndif_,npa)    

In [ ]:
ndif = []

npa = []

for s in range(0,len(raw_int.columns)):
    npa.append([raw_int.columns[s]])
    
ndif = clientFunction(ndif_,npa)           

In [ ]:
#doesn't preserve na's...
#len(pmdarima.utils.diff(temp,1,1).ravel())

deseasoned = pd.DataFrame()
for i in range(0,len(raw_int.columns)):
    if(sndif[i]*season == 0):
        temp = raw_int.iloc[:,[i]]
    else:
        temp = raw_int.iloc[:,[i]]
        if(sndif[i]>0):
            for d in range(0,sndif[i]):
                temp = pd.DataFrame(temp.values.ravel()-lagpad(temp.values.ravel(),1*season)).set_index(temp.index)
                temp.columns = raw_int.iloc[:,[i]].columns
    deseasoned = pd.concat([deseasoned,temp],axis=1)


In [ ]:
#for d in deseasoned.columns:
    #print(d)

In [ ]:
deseasoned_differenced = pd.DataFrame()

for i in range(0,len(raw_int.columns)):
    temp_ = deseasoned.iloc[:,[i]]
    colnames = temp_.columns
    if ndif[i]>0:
        #print(ndif[i])
        for d in range(0,ndif[i]):
            #print(d)
            #
            #print(temp_.columns)
            #temp_ = pd.DataFrame(temp_.values.ravel()-lagpad(temp_.values.ravel(),1)).set_index(temp_.index)
            #temp_.columns = colnames
            temp = temp_.diff()
    temp.columns = temp_.columns
    deseasoned_differenced = pd.concat([deseasoned_differenced,temp],axis=1)


In [ ]:
#for d in deseasoned_differenced.columns:
    #print(d)

In [ ]:
deseasoned_differenced.interpolate(method='time').isna().sum().sum()
#.fillna(method='bfill')
#raw_int = raw.dropna()


In [ ]:
deseasoned_differenced.describe()

In [ ]:
#for d in deseasoned_differenced.columns:
    #print(d)

In [ ]:
#https://machinelearningmastery.com/time-series-data-stationary-python/
cleaned = deseasoned_differenced.interpolate(method='time').dropna()


In [ ]:
#plt.plot(cleaned.iloc[:,0])

In [ ]:
#How many are stationary?
%matplotlib inline
pd.DataFrame(cleaned.apply(adfuller).iloc[1,]).iloc[:,0].hist()

In [ ]:
#for d in cleaned.columns:
    #print(d)

In [ ]:
indexes = train_test_split(cleaned.index, test_size=.5, random_state=0, shuffle=False)

In [ ]:
test_sets = []

for i in indexes:
    test_sets.append(cleaned.index.difference(i))
    
training = indexes[0]
testing = indexes[1]

In [ ]:
#np.cumsum(x_names=='BOGZ1FL105015105Q')

In [ ]:
#len(cleaned.iloc[:,raw.columns=='BACDINA066MNFRBNY'])

In [ ]:
ccf_max_lag = 4

#for c in cleaned.columns:
    #print(y_name)


In [ ]:
ccf_ = []

npa = []

chosen = cleaned.columns[random.randint(0,len(cleaned.columns)-1)]
print(chosen)
y_name = cleaned.columns[cleaned.columns==chosen].values[0]
#x_names = cleaned.columns[(cleaned.columns!=cleaned.columns[0])]
x_names = cleaned.columns

for s in range(0,len(x_names)):
    #y_name = y_name_
    x_name = x_names[s]
    #print(x_name)
    npa.append([y_name,x_name])
    
ccf_ = clientFunction(ret_ccf,npa)

y = np.array(cleaned.iloc[:,cleaned.columns==y_name]).ravel()
x = y
#last one is for comparing with itsel to ensure 0 lag ccf is 1
ccf_.append([y_name,y_name,crosscorrelation(x,y, ccf_max_lag, mode='corr')])
#ccf_.append([np.array([y_name,y_name]).reshape(2,1),crosscorrelation(x,y, 4, mode='corr')])#


In [ ]:
%matplotlib inline
plt.plot(cleaned[chosen])

In [ ]:
range_ = [*range(-ccf_max_lag,ccf_max_lag+1)].copy()

ccf_scores = pd.DataFrame()

for c in ccf_:
    #print(c)
    y = c[0]
    x = c[1]
    #print(x)
    ar_ = pd.DataFrame(c[2])
    #print(ar_)
    ar_.index = range_
    ar_.columns = [x]
    #abs(ar_)
    ccf_scores = pd.concat([ccf_scores,ar_],axis=1)
    
    

In [ ]:
#len(ccf_scores.columns)

In [ ]:
#derive optimally lagged dataset

data_final = pd.DataFrame()

best_lags = []
#don't want the last one
for c in ccf_scores.columns[:-1]:
    temp = ccf_scores[ccf_scores.index>0][c]
    bl = ccf_scores.index[ccf_scores.index>0][np.argmax(abs(temp))]
    best_lags.append(bl)
    data = pd.DataFrame(lagpad(cleaned[c],bl))
    data.index = cleaned[c].index
    data.columns = [c]
    data_final = pd.concat([data_final,data],axis=1)


In [ ]:
data_final_dask = dd.from_pandas(data_final,npartitions=128)
data_final_dask_w_y = dd.concat([cleaned[[y_name]],data_final_dask.compute()],axis=1)
names_ = ['target']
names_.extend(cleaned.columns)
data_final_dask_w_y.columns = names_

In [ ]:
#data_final_dask.apply(np.cumsum,axis=1).compute()


In [ ]:
len(data_final.columns)

In [ ]:
#plt.plot(raw_int[chosen])
#plt.plot(cleaned[chosen])
#plt.plot(data_final_dask_w_y[chosen].compute())
plt.plot(data_final_dask_w_y[['target']].compute().loc[training])

In [ ]:
'''
zca1_subset = working_set[subset.columns.difference(['target',max_pname])].dropna()
zca1 = zca.fit(zca1_subset)
zca1_df = pd.DataFrame(zca1.transform(zca1_subset))
zca1_df.columns = zca1_subset.columns
zca1_df.index = zca1_subset.index

zca1_subset = working_set[subset.columns.difference(['target',max_pname])].dropna()
zca1 = zca.fit(zca1_subset)
zca1_df = pd.DataFrame(zca1.transform(zca1_subset))
zca1_df.columns = zca1_subset.columns
zca1_df.index = zca1_subset.index
#pd.concat([target,zca1],axis=1)
'''

In [ ]:
chosen

In [ ]:
#New_Names[np.argmax(np.array(New_Names)==chosen)]

In [ ]:
#n = New_Names[3]
#New_Names==n

In [ ]:
#New_Names = (data_final_dask_w_y.columns.difference(['target']))


In [ ]:
subset_test

In [ ]:
subset_test['target']

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
tscv = TimeSeriesSplit(n_splits = 5)
rmse = []

both_ = []
#train_ = []
#test_ = []
for train_index, test_index in tscv.split(outer_dataset.index):
    #train_.append(train_index)
    #test_.append(test_index)
    both_.append([train_index,test_index])


In [276]:
def cv_split (dataset):
    both_ = []
    #train_ = []
    #test_ = []
    for train_index, test_index in tscv.split(outer_dataset.index):
        #train_.append(train_index)
        #test_.append(test_index)
        both_.append([train_index,test_index])    
    return(both_)

def cv_check (npa_):
    dataset= outer_dataset
    train_index = npa_[0]
    #print(train_index)
    test_index = npa_[1]
    #I don't need it to do training/test splits, but I had advanced ideas that would apply linear models to a test partition and go with the best error reduction... 
    # but partial correlations are just that except they don't take into consideration training/test partitions

    #target.iloc[training].iloc[train_index]
    subset_train = dataset.iloc[train_index]
    subset_test = dataset.iloc[test_index]

    dist = scipy.stats.beta(n_/2 - 1, n_/2 - 1, loc=-1, scale=2)

    y_reg_train_no_x = LinearRegression().fit(subset_train[New_Names_testing], target.iloc[train_index])
    y_fore_no_x = y_reg_train_no_x.predict(subset_test[New_Names_testing])
    y_resid_no_x = y_fore_no_x.ravel()-subset_test['target']

    x_reg_train_no_x = LinearRegression().fit(subset_train[New_Names_testing], target.iloc[train_index])
    x_fore_no_x = x_reg_train_no_x.predict(subset_test[New_Names_testing])
    x_resid_no_x = x_fore_no_x.ravel()-subset_test[n]

    cor_resid = pd.concat([pd.DataFrame(y_resid_no_x),pd.DataFrame(x_resid_no_x)],axis=1).corr()
    #model_name = ols(formula_from_cols(subset, 'target'),data=data_final_dask_w_y[subset.columns].compute().iloc[train_index]).fit()
    #print(model_name.summary())

    #skip y and states
    #set_ = subset.loc[:, ~subset.columns.isin([target])].columns.tolist()

    c_value = np.array(cor_resid).ravel()[1]

    return(c_value)



[0.4314668649612061,
 0.14602706807003135,
 0.9194595627649033,
 0.9996928696052728,
 0.9970097928413545]

In [280]:
scaler = StandardScaler()

tscv = TimeSeriesSplit(n_splits = 5)

#scaler.fit(np.array(data_final_dask_w_y[['target']].compute().loc[training]).reshape(-1, 1))

New_Names = list(data_final_dask_w_y.columns.difference(['target']))

outer_dataset = data_final_dask_w_y.compute().loc[training].dropna()
target = outer_dataset[['target']]

subset = pd.concat([target,outer_dataset[New_Names]],axis=1)

num_folds = 2
#kfold = KFold(n_splits=num_folds, shuffle=False)
#train, test = kfold.get_n_splits(outer_dataset.index)

p_threshold = .05

iteration = 0
max_pvalue = 1

while(max_pvalue>=.05):
    #
    n_p_values = pd.DataFrame()
    
    #parallelize here (x16)
    for n in New_Names:

        New_Names_testing = list(np.array(New_Names)[(np.array(New_Names)!=n)])
        
        p_values = pd.DataFrame()
        #inner_c_values = []
        
        inner_c_values = clientFunction(cv_check,cv_split(outer_dataset))   
   
        n_ = len(subset_test) 

        p_value = 2*dist.cdf(-abs(np.mean(inner_c_values)))
        temp = pd.DataFrame([chosen,n,p_value]).T
        temp.columns = ['target','test','p']

        p_values = pd.concat([p_values,temp],axis=0)

        #print(p_values)
        #p_values_train = 
        #p_values_test = pd.DataFrame(2*dist.cdf(-abs(subset_test.pcorr()['target']))).T

        #p_values = (p_values_train + p_values_test)/2
        #p_values.columns = list(subset.columns)

        #max_pname = p_values.idxmax(axis=1)[0]
        #max_pvalue = p_values[max_pname].values[0]

        #check correlation of residuals
        #print(p_values)
    n_p_values = pd.concat([n_p_values,p_values],axis=0)
    print(n_p_values)
    '''
    if (max_pvalue > .05):
        print([max_pname, max_pvalue])
        #New_Names.remove(max_pname)
        New_Names_testing = list(np.array(New_Names_testing)[(np.array(New_Names_testing)!=max_pname)])
        temp = ['target']
        temp.extend(New_Names_testing)
        subset = subset[temp]
    '''

        

KeyboardInterrupt: 

In [ ]:
data_temp = data_final_dask_w_y.compute()

In [ ]:
subset[New_Names]

In [ ]:
len(New_Names)

In [ ]:
model_name= ols(formula_from_cols(data_temp[New_Names], 'target'),data=data_temp)

In [ ]:
print(model_name.summary())

In [ ]:
'''
includes = []
for c in subset.columns:
    index = np.argwhere(data_final_dask_w_y.columns==c)[0][0]
    includes.append(index)
'''

In [ ]:
%matplotlib inline
corrMatrix = data_final_dask_w_y[subset.columns].compute().loc[training].dropna().pcorr().sort_values(kind="quicksort", by='target', ascending=False,key=abs)
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
#reg = train(data_final_dask_w_y[subset.columns].compute().loc[training].dropna())
#subset.dropna()


In [ ]:
reg = regress(data_final_dask_w_y[subset.columns].compute().dropna())

In [ ]:
variables = subset.iloc[training]#subset.columns[1:]
#print(variables[f])
target=variables['target']
fig = plot_sfs(reg, kind='std_err')
plt.title('Sequential Forward Selection (w. StdErr)')
#plt.savefig(str(target)+'.png', dpi=300, format='png', bbox_inches='tight')
plt.grid()
plt.show()

In [ ]:

model_name = ols(formula_from_cols(subset, 'target'),data=data_final_dask_w_y[subset.columns].compute().dropna()).fit()
model_name.summary()